In [9]:
import torch
import torch.nn as nn
from model import Transformer
from config import get_config, get_weights_file_path
from train import get_model, get_ds, greedy_decode
import altair as alt
import pandas as pd
import numpy as np
import os

In [10]:
# Define The Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using Device: {device}")

Using Device: cuda


In [11]:
config = get_config()
train_dataloader, val_dataloader, vocab_src, vocab_tgt = get_ds(config)
model = get_model(config, vocab_src.get_vocab_size(), vocab_tgt.get_vocab_size()).to(device)

# Load The Pretrained Weights
model_filename = get_weights_file_path(config, f"06")
state = torch.load(model_filename)
model.load_state_dict(state['model_state_dict'])

Filtering long sentences: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1659083/1659083 [00:25<00:00, 65014.50it/s]


Max Length Of Source Sentence: 36
Max Length Of Target Sentence: 106


<All keys matched successfully>

In [12]:
def load_next_batch():
    
    batch = next(iter(val_dataloader))
    encoder_input = batch['encoder_input'].to(device)
    encoder_mask = batch['encoder_mask'].to(device)
    decoder_input = batch['decoder_input'].to(device)
    decoder_mask = batch['decoder_mask'].to(device)
    
    encoder_input_tokens = [vocab_src.id_to_token(idx) for idx in encoder_input[0].cpu().numpy()]
    decoder_input_tokens = [vocab_tgt.id_to_token(idx) for idx in decoder_input[0].cpu().numpy()]
    
    model_out = greedy_decode(model,
                              encoder_input,
                              encoder_mask,
                              vocab_src,
                              vocab_tgt,
                              config['seq_len'],
                              device)
    
    return batch, encoder_input_tokens, decoder_input_tokens

In [13]:
def mtx2df(m, 
           max_row, 
           max_col, 
           row_tokens, 
           col_tokens):
    
    return pd.DataFrame(
        [
            (
                r,
                c,
                float(m[r, c]),
                "%.3d %s" % (r, row_tokens[r] if len(row_tokens) > r else "<blank>"),
                "%.3d %s" % (c, col_tokens[c] if len(col_tokens) > c else "<blank>"),
            )
            for r in range(m.shape[0])
            for c in range(m.shape[1])
            if r < max_row and c < max_col
        ],
        columns=["row", "column", "value", "row_token", "col_token"],
    )

def get_attn_map(attn_type: str,
                 layer: int, 
                 head: int):
    
    if attn_type == "encoder":
        attn = model.encoder.layers[layer].self_attention_block.attention_scores
    elif attn_type == "decoder":
        attn = model.decoder.layers[layer].self_attention_block.attention_scores
    elif attn_type == "encoder-decoder":
        attn = model.decoder.layers[layer].cross_attention_block.attention_scores
    return attn[0, head].data

def attn_map(attn_type,
             layer, 
             head, 
             row_tokens,
             col_tokens, 
             max_sentence_len):
    
    df = mtx2df(
        get_attn_map(attn_type, 
                     layer, 
                     head),
        max_sentence_len,
        max_sentence_len,
        row_tokens,
        col_tokens,
    )
    return (
        alt.Chart(data=df)
        .mark_rect()
        .encode(
            x=alt.X("col_token", axis=alt.Axis(title="")),
            y=alt.Y("row_token", axis=alt.Axis(title="")),
            color="value",
            tooltip=["row", "column", "value", "row_token", "col_token"],
        )
        #.title(f"Layer {layer} Head {head}")
        .properties(height=400, width=400, title=f"Layer {layer} Head {head}")
        .interactive()
    )

# --- Updated Function with Output Path ---
def get_all_attention_maps(attn_type: str,
                           layers: list[int],
                           heads: list[int],
                           row_tokens: list,
                           col_tokens,
                           max_sentence_len: int,
                           save_path: str = "attention_plots"): # New parameter
    
    # 1. Create the output directory if it doesn't exist
    if not os.path.exists(save_path):
        os.makedirs(save_path)
        print(f"Created directory: {save_path}")

    charts = []
    for layer in layers:
        rowCharts = []
        for head in heads:
            rowCharts.append(attn_map(attn_type, layer, head, row_tokens, col_tokens, max_sentence_len))
        charts.append(alt.hconcat(*rowCharts))
        
    viz = alt.vconcat(*charts)
    
    # 2. Construct the full file path
    file_path = os.path.join(save_path, f'{attn_type}.png')
    
    # 3. Save to the specific path
    viz.save(file_path)
    print(f"Saved plot to: {file_path}")
    
    return viz

In [14]:
batch, encoder_input_tokens, decoder_input_tokens = load_next_batch()
print(f'Source: {batch["src_text"][0]}')
print(f'Target: {batch["tgt_text"][0]}')
sentence_len = encoder_input_tokens.index('[PAD]')

Source: “What 's going to happen next? ”“
Target: अब क्या होगा? ”


In [15]:
# Define your output folder
OUTPUT_DIR = "results/visualizations"

# ... (Load your batch and tokens as before) ...

# Select meaningful layers (first and last)
selected_layers = [0, len(model.encoder.layers) - 1]
selected_heads = [0, 1, 2, 3, 4, 5, 6, 7]

# 1. Decoder Self-Attention
get_all_attention_maps("decoder", 
                       selected_layers, 
                       selected_heads, 
                       decoder_input_tokens, 
                       decoder_input_tokens, 
                       min(20, sentence_len),
                       save_path=OUTPUT_DIR) # Pass the path here

# 2. Cross-Attention (Remember the swapped tokens fix from before!)
get_all_attention_maps("encoder-decoder", 
                       selected_layers, 
                       selected_heads,
                       decoder_input_tokens, # Rows = Decoder
                       encoder_input_tokens, # Cols = Encoder
                       min(20, sentence_len),
                       save_path=OUTPUT_DIR) # Pass the path here

C:\Users\Kar_a\AppData\Local\Temp\ipykernel_27496\4033266025.py:11: UserWarning: Automatically deduplicated selection parameter with identical configuration. If you want independent parameters, explicitly name them differently (e.g., name='param1', name='param2'). See https://github.com/vega/altair/issues/3891
  get_all_attention_maps("decoder",
C:\Users\Kar_a\AppData\Local\Temp\ipykernel_27496\4033266025.py:11: UserWarning: Automatically deduplicated selection parameter with identical configuration. If you want independent parameters, explicitly name them differently (e.g., name='param1', name='param2'). See https://github.com/vega/altair/issues/3891
  get_all_attention_maps("decoder",
C:\Users\Kar_a\AppData\Local\Temp\ipykernel_27496\4033266025.py:11: UserWarning: Automatically deduplicated selection parameter with identical configuration. If you want independent parameters, explicitly name them differently (e.g., name='param1', name='param2'). See https://github.com/vega/altair/issu

Saved plot to: results/visualizations\decoder.png


C:\Users\Kar_a\AppData\Local\Temp\ipykernel_27496\4033266025.py:20: UserWarning: Automatically deduplicated selection parameter with identical configuration. If you want independent parameters, explicitly name them differently (e.g., name='param1', name='param2'). See https://github.com/vega/altair/issues/3891
  get_all_attention_maps("encoder-decoder",
C:\Users\Kar_a\AppData\Local\Temp\ipykernel_27496\4033266025.py:20: UserWarning: Automatically deduplicated selection parameter with identical configuration. If you want independent parameters, explicitly name them differently (e.g., name='param1', name='param2'). See https://github.com/vega/altair/issues/3891
  get_all_attention_maps("encoder-decoder",
C:\Users\Kar_a\AppData\Local\Temp\ipykernel_27496\4033266025.py:20: UserWarning: Automatically deduplicated selection parameter with identical configuration. If you want independent parameters, explicitly name them differently (e.g., name='param1', name='param2'). See https://github.com/

Saved plot to: results/visualizations\encoder-decoder.png


alt.VConcatChart(...)